# AI Resume Analyzer

This project is a tool for analyzing resumes using natural language processing (NLP) techniques. The goal is to help recruiters and hiring managers quickly identify the most qualified candidates for a job opening. The tool extracts skills and qualifications from a resume, and matches them against a list of desired skills provided by the user.


# Code Explanation

The code is divided into several blocks, each performing a specific task:

- Block 1: Imports the required libraries and packages.
- Block 2: Loads the NLP model and defines a function for extracting skills from a resume.
- Block 3: Reads the resume file and preprocesses the data.
- Block 4: Extracts the skills from the resume and creates a TF-IDF matrix.
- Block 5: Trains a nearest neighbors model on the TF-IDF matrix.
- Block 6: Defines a function for finding the best match for a given set of skills.
- Block 7: Runs the function on the test data and displays the results.


# Required Libraries and Packages

In [1]:
from pyresparser import ResumeParser
import os
from docx import Document
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [2]:
df =pd.read_csv('jobs.csv') 
df

,Unnamed: 0,url,Position,Company,Location,Job_Description
0,0,https://www.glassdoor.co.in/partner/jobListing...,Software Testing Internship,Smart Food Safe Solutions Inc,– Bengaluru,About the company:\nSmart Food Safe Solutions ...
1,1,https://www.glassdoor.co.in/partner/jobListing...,Embedded Software Testing,Mobiveil,– Bengaluru,Location : Bangalore\nExperience : 4+ Years\n\...
2,2,https://www.glassdoor.co.in/partner/jobListing...,Senior Engineer - Software Testing (Bangalore ...,Open Systems International,– Bengaluru,"Open Systems International, Inc. (OSI) www.osi..."
3,3,https://www.glassdoor.co.in/partner/jobListing...,Software Testing Engineer,Bloom Solutions,– Bengaluru,About the Job\n\nSoftware Testing Engineer\n\n...
4,4,https://www.glassdoor.co.in/partner/jobListing...,CIEL/SEL/1888: Software testing Engineer,CIEL HR Services,– Bengaluru,Location: Bangalore\nExperience: 3 to 6Years\n...
...,...,...,...,...,...,...
1919,3113,https://www.glassdoor.co.in/partner/jobListing...,Front End Developer,Cuemath,Bengaluru,Skills and Qualifications:\n\n2+ Years of expe...
1920,3115,https://www.glassdoor.co.in/partner/jobListing...,Technology Lead-Sharepoint Developer,Infogain,Bengaluru,Job ID : TH10519_13189\n\nPosted on: 29th of M...
1921,3120,https://www.glassdoor.co.in/partner/jobListing...,Senior UI Developer,Siemens PLC,Bengaluru,Job Description\nWe spend 90 percent of\n\nour...
1922,3122,https://www.glassdoor.co.in/partner/jobListing...,Web Developer,Fidelity Investments,Bengaluru,(Job Number: 1905027)\n\nJob Title â€“ Web Dev...


In [3]:
stopw  = set(stopwords.words('english'))
df['test']=df['Job_Description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>2 and word not in (stopw)]))
df['test']

0       About company: Smart Food Safe Solutions Inc. ...
1       Location Bangalore Experience Years Job Descri...
2       Open Systems International, Inc. (OSI) www.osi...
3       About Job Software Testing Engineer Job Descri...
4       Location: Bangalore Experience: 6Years Skills ...
                              ...                        
1919    Skills Qualifications: Years experience Strong...
1920    Job TH10519_13189 Posted on: 29th May, 2019Job...
1921    Job Description spend percent lives buildings....
1922    (Job Number: 1905027) Job Title â€“ Web Develo...
1923    marry design engineering language ways produce...
Name: test, Length: 1924, dtype: object

In [4]:
import spacy

nlp = spacy.load('en_core_web_sm')

# File Format

The tool supports three file formats for resumes: .txt, .docx, and .pdf. The file should contain the candidate's name, contact information, education, work experience, and skills.


In [5]:
# Prompt the user to specify the file path of a resume in .txt, .docx, or .pdf format
filed = input('Specify the path of the resume (format(.txt, .docx and .pdf)): ')

try:
    # If the file is not in .txt format, create a new Word document, add its contents, save as .docx and extract from there
    doc = Document()
    with open(filed, 'r') as file:
        doc.add_paragraph(file.read())
    doc.save("text.docx")
    data1 = ResumeParser('text.docx').get_extracted_data()

except:
    # If the file is in .txt format, extract the contents directly
    data1 = ResumeParser(filed).get_extracted_data()
    
# Extract the 'skills' section from the extracted data and store it in the 'resume' variable
resume = data1['skills']

Specify the path of the resume (format(.txt, .docx and .pdf)): resume.pdf


C:\Users\user\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [6]:
print(resume)

['Javascript', 'Ibm', 'Python', 'Technical', 'Coding', 'Api', 'English', 'Machine learning', 'Aws', 'Engineering', 'Video', 'Audio', 'C++', 'Algorithms', 'Html', 'Css', 'C']


In [7]:
skills=[]
skills.append(' '.join(word for word in resume))
skills

['Javascript Ibm Python Technical Coding Api English Machine learning Aws Engineering Video Audio C++ Algorithms Html Css C']

In [8]:
import re
from ftfy import fix_text

def ngrams(string, n=3):
    # fix any Unicode encoding issues
    string = fix_text(string)
    # remove non-ASCII characters
    string = string.encode("ascii", errors="ignore").decode()
    # convert to lowercase
    string = string.lower()
    # remove specified characters using regex
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    # replace '&' with 'and'
    string = string.replace('&', 'and')
    # replace ',' with a space
    string = string.replace(',', ' ')
    # replace '-' with a space
    string = string.replace('-', ' ')
    # normalise case - capital at start of each word
    string = string.title()
    # replace multiple spaces with a single space
    string = re.sub(' +',' ',string).strip()
    # pad names for ngrams
    string = ' '+ string +' '
    # remove certain characters
    string = re.sub(r'[,-./]|\sBD',r'', string)
    # create n-grams
    ngrams = zip(*[string[i:] for i in range(n)])
    # concatenate the n-grams into strings and return them as a list
    return [''.join(ngram) for ngram in ngrams]


In [9]:
# Import the TfidfVectorizer class from scikit-learn's feature extraction module
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the vectorizer object with minimum document frequency of 1 and ngram analyzer
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)

# Use the fit_transform method to learn the vocabulary dictionary and return a term-document matrix
# The input is the list of skills, which will be transformed into a matrix of TF-IDF features
tfidf = vectorizer.fit_transform(skills)

# The resulting tfidf matrix is a sparse matrix of shape (n_skills, n_features)
# Each row represents a skill and each column represents an n-gram feature in the skill text

# Implementing NearestNeighbors

In [10]:
# Import the NearestNeighbors class from scikit-learn's neighbors module
from sklearn.neighbors import NearestNeighbors

# Define the nearest neighbors object with k=1 (i.e., find the nearest neighbor) and use all available CPUs for parallelism
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

# Define the input data for the nearest neighbors search
# Here, the input data is the preprocessed and filtered text in the 'test' column of the dataframe 'df'
test = (df['test'].values.astype('U'))

In [11]:
# Define a function to get the nearest neighbor(s) of a given query
def getNearestN(query):
    # Transform the query text into a TF-IDF vector using the same vectorizer used for the skills
    queryTFIDF_ = vectorizer.transform(query)
    # Use the fitted nearest neighbors model to find the nearest neighbor(s) of the query TF-IDF vector
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    # Return the distances and indices of the nearest neighbor(s)
    return distances, indices

In [12]:
# Use the getNearestN function to find the nearest neighbor(s) of each job description in the 'test' array
distances, indices = getNearestN(test)

# Convert the 'test' array to a list
test = list(test) 

# Create an empty list to store the match confidence scores
matches = []

# Loop over the indices of the nearest neighbor(s) for each job description
for i,j in enumerate(indices):
    # Get the distance between the job description and its nearest neighbor(s)
    dist = round(distances[i][0], 2)
    
    # Create a temporary list to store the match confidence score for the current job description
    temp = [dist]
    
    # Append the match confidence score to the 'matches' list
    matches.append(temp)
    
# Convert the 'matches' list to a pandas DataFrame with a column named 'Match confidence'
matches = pd.DataFrame(matches, columns=['Match confidence'])
matches

,Match confidence
0,0.90
1,1.02
2,0.90
3,0.99
4,0.94
...,...
1919,1.01
1920,0.82
1921,0.90
1922,0.90


In [13]:
df['match']=matches['Match confidence']
df1=df.sort_values('match')
df1[['Position', 'Company','Location']].head(10).reset_index()

,index,Position,Company,Location
0,853,Data Scientist - Centre of Excellence,Micro Focus,â€“ Bengaluru
1,1268,Programmer Analyst - Data Science and Machine ...,FIS,Bengaluru
2,822,Machine Learning Engineer II,American Express,â€“ Bengaluru
3,1327,Machine Learning Engineer,American Express,Bengaluru
4,1328,Principal Machine Learning Engineer,Bengaluru,Bengaluru
5,1169,Data Scientist,Alphonso,Bengaluru
6,378,"Senior Advisor, Machine Learning Engineer (I8)",Dell,– Bengaluru
7,772,Data Scientist,ITC Infotech India Ltd,â€“ Bengaluru
8,142,"Data Science Engineer (ML, AI, Python) - 4 to ...",Capgemini,– Pune
9,1817,Front-End Engineer,Amazon,Bengaluru


# Implementing NearestNeighbors using cosine similarity

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
# Compute cosine similarity matrix for the tfidf matrix
cosine_similarities = cosine_similarity(tfidf, tfidf)

# Define the nearest neighbors object with k=1 (i.e., find the nearest neighbor)
nbrs = NearestNeighbors(n_neighbors=1, metric='precomputed')

# Fit the nearest neighbors model to the cosine similarity matrix
nbrs.fit(cosine_similarities)

# Define the input data for the nearest neighbors search
# Here, the input data is the preprocessed and filtered text in the 'test' column of the dataframe 'df'
test = df['test'].values.astype('U')

In [16]:
def getNearestN(query):
    # Transform the query text into a TF-IDF vector using the same vectorizer used for the skills
    queryTFIDF_ = vectorizer.transform(query)
    # Compute cosine similarity between the query TF-IDF vector and all other TF-IDF vectors
    query_cosine_similarities = cosine_similarity(queryTFIDF_, tfidf)
    # Use the fitted nearest neighbors model to find the nearest neighbor(s) of the query
    distances, indices = nbrs.kneighbors(query_cosine_similarities)
    # Return the distances and indices of the nearest neighbor(s)
    return distances, indices

In [17]:
distances, indices = getNearestN(test)

# Convert the 'test' array to a list
test = list(test) 

# Create an empty list to store the match confidence scores
matches = []

# Loop over the indices of the nearest neighbor(s) for each job description
for i,j in enumerate(indices):
    # Get the distance between the job description and its nearest neighbor(s)
    dist = round(distances[i][0], 2)
    
    # Create a temporary list to store the match confidence score for the current job description
    temp = [dist]
    
    # Append the match confidence score to the 'matches' list
    matches.append(temp)
    
# Convert the 'matches' list to a pandas DataFrame with a column named 'Match confidence'
matches = pd.DataFrame(matches, columns=['Match confidence'])
matches

,Match confidence
0,0.59
1,0.48
2,0.59
3,0.51
4,0.56
...,...
1919,0.49
1920,0.66
1921,0.60
1922,0.60


In [18]:
df['match']=matches['Match confidence']
df1=df.sort_values('match')
df1[['Position', 'Company','Location']].head(10).reset_index()

,index,Position,Company,Location
0,686,Lead - Global Advanced Analytics Data Scientist-2,Diageo,â€“ Bengaluru
1,826,Data Scientist (Contract),Tookitaki Holding PTE LTD,â€“ Bengaluru
2,1312,Data Analyst,Enquero,Bengaluru
3,410,Senior Software Engineer/Lead Data Engineer,Photon Infotech,– Chennai
4,920,Data Analyst,Xpheno,Bengaluru
5,641,Data Engineer,Knoema,â€“ Bengaluru
6,542,Web Developer .NET,"Netrix, LLC",– Kochi
7,385,Data Science Lead,MetricStream,– Bengaluru
8,1143,Data Scientist â€“ 9 to 12 years â€“,Capgemini,Bengaluru
9,1026,Data Scientist - Early Career Program,Ericsson-Worldwide,Bengaluru


# Implementing KNeighbors

In [19]:
from sklearn.neighbors import KNeighborsClassifier

In [20]:
knn = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
knn.fit(tfidf, [0]*tfidf.shape[0])
test = (df['test'].values.astype('U'))

In [21]:
def getNearestN(query):
    # Transform the query text into a TF-IDF vector using the same vectorizer used for the skills
    queryTFIDF_ = vectorizer.transform(query)
    # Use the fitted k-nearest neighbors model to find the nearest neighbor(s) of the query TF-IDF vector
    distances, indices = knn.kneighbors(queryTFIDF_)
    # Return the distances and indices of the nearest neighbor(s)
    return distances, indices

In [22]:
distances, indices = getNearestN(test)
test = list(test) 
# Loop over the indices of the nearest neighbor(s) for each job description
for i,j in enumerate(indices):
    # Get the distance between the job description and its nearest neighbor(s)
    dist = round(distances[i][0], 2)
    
    # Create a temporary list to store the match confidence score for the current job description
    temp = [dist]
    
    # Append the match confidence score to the 'matches' list
    matches.append(temp)

In [23]:
matches = pd.DataFrame(matches, columns=['Match confidence'])
matches

,Match confidence
0,0.59
1,0.48
2,0.59
3,0.51
4,0.56
...,...
1919,0.49
1920,0.66
1921,0.60
1922,0.60


In [24]:
df['match']=matches['Match confidence']
df1=df.sort_values('match')
df1[['Position', 'Company','Location']].head(10).reset_index()

,index,Position,Company,Location
0,686,Lead - Global Advanced Analytics Data Scientist-2,Diageo,â€“ Bengaluru
1,826,Data Scientist (Contract),Tookitaki Holding PTE LTD,â€“ Bengaluru
2,1312,Data Analyst,Enquero,Bengaluru
3,410,Senior Software Engineer/Lead Data Engineer,Photon Infotech,– Chennai
4,920,Data Analyst,Xpheno,Bengaluru
5,641,Data Engineer,Knoema,â€“ Bengaluru
6,542,Web Developer .NET,"Netrix, LLC",– Kochi
7,385,Data Science Lead,MetricStream,– Bengaluru
8,1143,Data Scientist â€“ 9 to 12 years â€“,Capgemini,Bengaluru
9,1026,Data Scientist - Early Career Program,Ericsson-Worldwide,Bengaluru


# Implementing KNeighbors using cosine similarity

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
# Compute cosine similarity matrix for the tfidf matrix
cosine_similarities = cosine_similarity(tfidf, tfidf)

# Instantiate the KNeighborsClassifier with the desired number of neighbors
knn = KNeighborsClassifier(n_neighbors=1, metric='precomputed')

# Fit the KNN model to the cosine similarity matrix
knn.fit(cosine_similarities, [0]*cosine_similarities.shape[0])

# Define the input data for the nearest neighbors search
# Here, the input data is the preprocessed and filtered text in the 'test' column of the dataframe 'df'
test = df['test'].values.astype('U')


In [39]:
def getNearestN(query):
    # Transform the query text into a TF-IDF vector using the same vectorizer used for the skills
    queryTFIDF_ = vectorizer.transform(query)
    # Compute cosine similarity between the query TF-IDF vector and all other TF-IDF vectors
    query_cosine_similarities = cosine_similarity(queryTFIDF_, tfidf)
    # Use the fitted nearest neighbors model to find the nearest neighbor(s) of the query
    distances, indices = knn.kneighbors(query_cosine_similarities)
    # Return the distances and indices of the nearest neighbor(s)
    return distances, indices

In [40]:
distances, indices = getNearestN(test)
test = list(test) 
# Loop over the indices of the nearest neighbor(s) for each job description
for i,j in enumerate(indices):
    # Get the distance between the job description and its nearest neighbor(s)
    dist = round(distances[i][0], 2)
    
    # Create a temporary list to store the match confidence score for the current job description
    temp = [dist]
    
    # Append the match confidence score to the 'matches' list
    matches.append(temp)

In [41]:
# Convert the 'matches' list to a pandas DataFrame with a column named 'Match confidence'
matches = pd.DataFrame(matches, columns=['Match confidence'])
matches

,Match confidence
0,0.59
1,0.48
2,0.59
3,0.51
4,0.56
...,...
1919,0.49
1920,0.66
1921,0.60
1922,0.60


In [42]:
df['match']=matches['Match confidence']
df1=df.sort_values('match')
df1[['Position', 'Company','Location']].head(10).reset_index()

,index,Position,Company,Location
0,686,Lead - Global Advanced Analytics Data Scientist-2,Diageo,â€“ Bengaluru
1,826,Data Scientist (Contract),Tookitaki Holding PTE LTD,â€“ Bengaluru
2,1312,Data Analyst,Enquero,Bengaluru
3,410,Senior Software Engineer/Lead Data Engineer,Photon Infotech,– Chennai
4,920,Data Analyst,Xpheno,Bengaluru
5,641,Data Engineer,Knoema,â€“ Bengaluru
6,542,Web Developer .NET,"Netrix, LLC",– Kochi
7,385,Data Science Lead,MetricStream,– Bengaluru
8,1143,Data Scientist â€“ 9 to 12 years â€“,Capgemini,Bengaluru
9,1026,Data Scientist - Early Career Program,Ericsson-Worldwide,Bengaluru


# Output

The output of the tool is a list of matches between the candidate's skills and the desired skills. Each match is assigned a confidence score, indicating how closely the candidate's skills match the desired skills. The matches are sorted in descending order of confidence score.


# Conclusion

In this project, we have developed a tool for analyzing resumes using NLP techniques. The tool can extract skills and qualifications from a resume, and match them against a list of desired skills. The tool has the potential to save recruiters and hiring managers time and effort by quickly identifying the most qualified candidates for a job opening.
